# Extract, Transform and Load

### Importing libraries

In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")

### Loading the dataset and visualizing the first rows to understand its structure.

In [150]:
df_wines= pd.read_csv('data/wine_reviews.csv')

In [151]:
df_wines.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In this first observation we can see that there is missing data

In [152]:
#dataframe structure
df_wines.shape

(129971, 14)

We have 129971 rows and 14 columns.

### We look at the types of variables we have

In [153]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [154]:
df_wines.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

### We check nulls and duplicate files

In [155]:
# check for null rows
print(df_wines.isnull().sum())

Unnamed: 0                   0
country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64


Among the columns with the most missing values are region_1, region_2, designation , taster_twitter_handle  and taster_name. 

In [156]:
# check for duplicated rows
sum(df_wines.duplicated())

0

There are no duplicates

### We make decisions with the columns in which it is important to correct their null value

In [157]:
# Calculate the percentage of remaining records if missing countries are dropped:
total_records = len(df_wines)
missing_countries = df_wines["country"].isnull().sum()
missing_ratio = 100 - ((missing_countries / total_records) * 100)
print("Missing countries: {}".format(missing_countries)+ "\n" +
      "Total records: {}".format(total_records) + "\n" + 
      "Percentage remaining: {:.2f}%".format(missing_ratio))

Missing countries: 63
Total records: 129971
Percentage remaining: 99.95%


Let's drop the 63 rows that are missing the Country value. They represent a very small sample of the dataset, and won't affect the results.

In [158]:
# Drop the rows with null values for "Country", update the reviews_df dataframe:
df_wines = df_wines.dropna(how="any", subset=["country", "variety"])

In [159]:
# Confirm null values have been dropped:
df_wines.shape

(129907, 14)

In [160]:
# Display confirmation of no records with "Country" null values:
df_wines[df_wines.country.isnull()]

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery


Impute null values ​​in 'Price' column with median, so as not to lose the records

In [161]:
median_price = df_wines['price'].median()
df_wines['price'].fillna(median_price, inplace=True)

In [162]:
# Verify that null values ​​have been imputed
print(df_wines.isnull().sum())

Unnamed: 0                   0
country                      0
description                  0
designation              37453
points                       0
price                        0
province                     0
region_1                 21183
region_2                 79396
taster_name              26243
taster_twitter_handle    31212
title                        0
variety                      0
winery                       0
dtype: int64


### We eliminate the columns that are not going to add value to our analysis and have a lot of missing data

In [163]:
# drop the missing columns
df_wines = df_wines.drop(["Unnamed: 0", "designation", "region_1", "region_2", "taster_name", "taster_twitter_handle"], axis=1)
df_wines.head()

,country,description,points,price,province,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,25.0,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [164]:
df_wines.shape

(129907, 8)

In [165]:
# Verify that null values ​​have been imputed
print(df_wines.isnull().sum())

country        0
description    0
points         0
price          0
province       0
title          0
variety        0
winery         0
dtype: int64


### We save the CSV file clean.

In [166]:
df_wines.to_csv('data/wines_clean.csv', index=False) 